In [1]:
#!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('../../../data/imdb_movies.csv')

C:\Users\danie\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,Le lion,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,...,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,$ 3507171,NaN,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,...,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,$ 7299062,NaN,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,2019-03-15,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,...,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",NaN,6.4,194,NaN,NaN,$ 2833,NaN,NaN,NaN


In [5]:
df = df.replace({'\$':''}, regex = True)

In [6]:
df.dtypes

imdb_title_id             object
title                     object
original_title            object
year                      object
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
dtype: object

In [7]:
df = df.dropna()

In [8]:
df

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
506,tt0017136,Metropolis,Metropolis,1927,1928-10-01,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,...,"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",In a futuristic city sharply divided between t...,8.3,156076,DEM 6000000,1236166,1349711,98.0,495.0,208.0
1048,tt0021749,Luci della città,City Lights,1931,1931-04-02,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,...,"Virginia Cherrill, Florence Lee, Harry Myers, ...","With the aid of a wealthy erratic tippler, a d...",8.5,162668,1500000,19181,46008,99.0,295.0,122.0
2454,tt0027977,Tempi moderni,Modern Times,1936,1937-03-12,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,...,"Charles Chaplin, Paulette Goddard, Henry Bergm...",The Tramp struggles to live in modern industri...,8.5,211250,1500000,163577,457688,96.0,307.0,115.0
2795,tt0029453,Il bandito della Casbah,Pépé le Moko,1937,1937-10-22,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,...,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fe...",A wanted gangster is both king and prisoner of...,7.7,6180,60000,155895,155895,98.0,46.0,55.0
2827,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938-11-30,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand",...,"Roy Atwell, Stuart Buchanan, Adriana Caselotti...",Exiled into the dangerous forest by her wicked...,7.6,177157,1499000,184925486,184925486,95.0,260.0,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85266,tt9208444,Impractical Jokers: The Movie,Impractical Jokers: The Movie,2020,2020-02-28,Comedy,92,USA,English,Chris Henchy,...,"Paula Abdul, Joe Gatto, James Murray, Brian Qu...",The story of a humiliating high school mishap ...,5.9,4105,3000000,10691091,10691091,39.0,152.0,17.0
85274,tt9214832,Emma.,Emma.,2020,2020-03-27,"Comedy, Drama",124,UK,English,Autumn de Wilde,...,"Anya Taylor-Joy, Angus Imrie, Letty Thomas, Ge...","In 1800s England, a well meaning but selfish y...",6.7,19858,10000000,10055355,25659965,71.0,314.0,188.0
85397,tt9354944,Jexi,Jexi,2019,2019-10-11,"Comedy, Romance",84,"USA, Canada",English,"Jon Lucas, Scott Moore",...,"Adam Devine, Alexandra Shipp, Rose Byrne, Ron ...",A comedy about what can happen when you love y...,6.1,17038,5000000,6546159,9341824,39.0,234.0,42.0
85460,tt9426210,Weathering with You,Tenki no ko,2019,2019-10-14,"Animation, Drama, Family",112,"Japan, China",Japanese,Makoto Shinkai,...,"Kotaro Daigo, Nana Mori, Shun Oguri, Sei Hirai...",A high-school boy who has run away to Tokyo be...,7.6,16277,11100000,7798743,193176979,72.0,177.0,110.0


In [9]:
df = df.drop(labels=506, axis=0)

In [10]:
df["date_published"] = pd.to_datetime(df.date_published, format="%Y/%m/%d")

In [11]:
df.index = df['imdb_title_id']

In [12]:
pd.set_option('display.max_columns', None)
df

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
imdb_title_id,,,,,,,,,,,,,,,,,,,,,,
tt0021749,tt0021749,Luci della città,City Lights,1931,1931-04-02,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, ...","With the aid of a wealthy erratic tippler, a d...",8.5,162668,1500000,19181,46008,99.0,295.0,122.0
tt0027977,tt0027977,Tempi moderni,Modern Times,1936,1937-03-12,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergm...",The Tramp struggles to live in modern industri...,8.5,211250,1500000,163577,457688,96.0,307.0,115.0
tt0029453,tt0029453,Il bandito della Casbah,Pépé le Moko,1937,1937-10-22,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,"Henri La Barthe, Henri La Barthe",Paris Film,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fe...",A wanted gangster is both king and prisoner of...,7.7,6180,60000,155895,155895,98.0,46.0,55.0
tt0029583,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938-11-30,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti...",Exiled into the dangerous forest by her wicked...,7.6,177157,1499000,184925486,184925486,95.0,260.0,173.0
tt0031381,tt0031381,Via col vento,Gone with the Wind,1939,1949-03-12,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh,...",A manipulative woman and a roguish man conduct...,8.1,283975,3977000,200852579,402352579,97.0,881.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9208444,tt9208444,Impractical Jokers: The Movie,Impractical Jokers: The Movie,2020,2020-02-28,Comedy,92,USA,English,Chris Henchy,"Chris Henchy, Joe Gatto",Funny or Die,"Paula Abdul, Joe Gatto, James Murray, Brian Qu...",The story of a humiliating high school mishap ...,5.9,4105,3000000,10691091,10691091,39.0,152.0,17.0
tt9214832,tt9214832,Emma.,Emma.,2020,2020-03-27,"Comedy, Drama",124,UK,English,Autumn de Wilde,"Eleanor Catton, Jane Austen",Working Title Films,"Anya Taylor-Joy, Angus Imrie, Letty Thomas, Ge...","In 1800s England, a well meaning but selfish y...",6.7,19858,10000000,10055355,25659965,71.0,314.0,188.0
tt9354944,tt9354944,Jexi,Jexi,2019,2019-10-11,"Comedy, Romance",84,"USA, Canada",English,"Jon Lucas, Scott Moore","Jon Lucas, Scott Moore",CBS Films,"Adam Devine, Alexandra Shipp, Rose Byrne, Ron ...",A comedy about what can happen when you love y...,6.1,17038,5000000,6546159,9341824,39.0,234.0,42.0


In [13]:
!pip install surprise

In [14]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate

reader = Reader()

data = Dataset.load_from_df(df[['imdb_title_id', 'title',"metascore"]], reader)

svd = SVD()

In [15]:
from surprise import BaselineOnly
cross_validate(BaselineOnly(), data, cv=5, verbose=True)


Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    53.4326 52.9403 53.2225 53.1695 52.5544 53.0639 0.2991  
MAE (testset)     50.4044 49.7574 50.0136 49.8269 49.4800 49.8964 0.3064  
Fit time          0.02    0.03    0.02    0.02    0.03    0.02    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([53.43261027, 52.94032494, 53.2225465 , 53.16948329, 52.55435943]),
 'test_mae': array([50.40438398, 49.75736961, 50.01360544, 49.82690854, 49.47996977]),
 'fit_time': (0.022939682006835938,
  0.025003910064697266,
  0.023936033248901367,
  0.02293682098388672,
  0.029894113540649414),
 'test_time': (0.007978677749633789,
  0.006989717483520508,
  0.007978677749633789,
  0.006981849670410156,
  0.006054878234863281)}

In [16]:
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV


In [17]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [18]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

In [19]:
# for Surprise, we only need three columns from the dataset
data = df[['imdb_title_id', 'title',"metascore"]]
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(data, reader=reader) #loads Pandas df 

# train-test-split
trainset, testset = train_test_split(data, test_size=.2)

In [20]:
svd = SVD() # default values
svd.fit(trainset)

In [21]:
predictions = svd.test(testset)
accuracy.rmse(predictions)

RMSE: 53.2313


53.2312728115704

In [22]:
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    53.3475 52.7277 53.1422 53.0156 53.0887 53.0643 0.2013  
MAE (testset)     50.1678 49.4603 50.0098 49.9690 49.8753 49.8964 0.2377  
Fit time          0.02    0.03    0.02    0.03    0.03    0.02    0.00    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([53.34747545, 52.72765492, 53.14220999, 53.01555698, 53.0887319 ]),
 'test_mae': array([50.16780045, 49.46031746, 50.00982615, 49.96900983, 49.87528345]),
 'fit_time': (0.02293848991394043,
  0.02586674690246582,
  0.024975299835205078,
  0.025099754333496094,
  0.025940656661987305),
 'test_time': (0.007089138031005859,
  0.007045269012451172,
  0.006981611251831055,
  0.007977962493896484,
  0.007969856262207031)}

In [23]:
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=5)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [24]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

benchmark = []
# Iterate over all algorithms

algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    # results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x000001C038E8BA30>, <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x000001C038E8BB50>, <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x000001C038E8B970>, <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x000001C038E8BBB0>, <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x000001C038E8BA90>, <surprise.prediction_algorithms.knns.KNNBaseline object at 0x000001C05768F070>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x000001C05768F910>, <surprise.prediction_algorithms.knns.KNNWithMeans object at 0x000001C05768F8B0>, <surprise.prediction_algorithms.knns.KNNWithZScore object at 0x000001C05768F8E0>, <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x000001C05768F550>, <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x000001C05768F580>] 



Starting:  <surprise.prediction_algorithms.

In [25]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [26]:
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
NMF,53.062566,0.978132,0.042886
SVDpp,53.063492,0.472073,0.012650
KNNWithZScore,53.063806,0.521414,0.014956
KNNBasic,53.063816,0.357376,0.014290
BaselineOnly,53.063888,0.020624,0.013632
KNNWithMeans,53.064243,0.407158,0.042875
SVD,53.064403,0.279584,0.013300
SlopeOne,53.064411,0.368024,0.013284
CoClustering,53.064479,1.084615,0.012965


In [27]:
param_grid = {
    "n_epochs": [10, 20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], refit=True, cv=5)

gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST MAE: \t", gs.best_score["mae"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 53.06315663989655
BEST MAE: 	 49.896447467876044
BEST params: 	 {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.02}


In [28]:
df.head(12)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
imdb_title_id,,,,,,,,,,,,,,,,,,,,,,
tt0021749,tt0021749,Luci della città,City Lights,1931,1931-04-02,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, ...","With the aid of a wealthy erratic tippler, a d...",8.5,162668,1500000,19181,46008,99.0,295.0,122.0
tt0027977,tt0027977,Tempi moderni,Modern Times,1936,1937-03-12,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergm...",The Tramp struggles to live in modern industri...,8.5,211250,1500000,163577,457688,96.0,307.0,115.0
tt0029453,tt0029453,Il bandito della Casbah,Pépé le Moko,1937,1937-10-22,"Crime, Drama, Romance",94,France,"French, Arabic",Julien Duvivier,"Henri La Barthe, Henri La Barthe",Paris Film,"Jean Gabin, Gabriel Gabrio, Saturnin Fabre, Fe...",A wanted gangster is both king and prisoner of...,7.7,6180,60000,155895,155895,98.0,46.0,55.0
tt0029583,tt0029583,Biancaneve e i sette nani,Snow White and the Seven Dwarfs,1937,1938-11-30,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti...",Exiled into the dangerous forest by her wicked...,7.6,177157,1499000,184925486,184925486,95.0,260.0,173.0
tt0031381,tt0031381,Via col vento,Gone with the Wind,1939,1949-03-12,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh,...",A manipulative woman and a roguish man conduct...,8.1,283975,3977000,200852579,402352579,97.0,881.0,197.0
tt0031679,tt0031679,Mr. Smith va a Washington,Mr. Smith Goes to Washington,1939,1947-04-05,"Comedy, Drama",129,USA,English,Frank Capra,"Sidney Buchman, Lewis R. Foster",Columbia Pictures,"Jean Arthur, James Stewart, Claude Rains, Edwa...",A naive man is appointed to fill a vacancy in ...,8.1,104547,1900000,144738,144738,73.0,296.0,88.0
tt0032138,tt0032138,Il mago di Oz,The Wizard of Oz,1939,1949-04-19,"Adventure, Family, Fantasy",102,USA,English,"Victor Fleming, George Cukor","Noel Langley, Florence Ryerson",Metro-Goldwyn-Mayer (MGM),"Judy Garland, Frank Morgan, Ray Bolger, Bert L...",Dorothy Gale is swept away from a farm in Kans...,8.0,366293,2777000,24790250,26142032,100.0,688.0,168.0
tt0032455,tt0032455,Fantasia,Fantasia,1940,1946-09-19,"Animation, Family, Fantasy",125,USA,English,"James Algar, Samuel Armstrong","Joe Grant, Dick Huemer",Walt Disney Productions,"Deems Taylor, Leopold Stokowski, The Philadelp...",A collection of animated interpretations of gr...,7.8,86795,2280000,76408097,76411401,96.0,342.0,119.0
tt0032910,tt0032910,Pinocchio,Pinocchio,1940,1947-11-27,"Animation, Comedy, Family",88,USA,English,"Norman Ferguson, T. Hee","Carlo Collodi, Ted Sears",Walt Disney Animation Studios,"Mel Blanc, Don Brodie, Stuart Buchanan, Walter...","A living puppet, with the help of a cricket as...",7.4,127618,2600000,84254167,121892045,99.0,202.0,140.0


In [29]:
from datetime import datetime
print(training_parameters)
reader = Reader(rating_scale=(1, 5))

print("\n\n\t\t STARTING\n\n")
start = datetime.now()

print("> Loading data...")
data = Dataset.load_from_df(df[['imdb_title_id', 'title',"metascore"]], reader)
print("> OK")

print("> Creating trainset...")
trainset = data.build_full_trainset()
print("> OK")


startTraining = datetime.now()
print("> Training...")

algo = SVD(n_epochs = training_parameters['n_epochs'], lr_all = training_parameters['lr_all'], reg_all = training_parameters['reg_all'])

algo.fit(trainset)

endTraining = datetime.now()
print("> OK \t\t It Took: ", (endTraining-startTraining).seconds, "seconds")


{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.02}


		 STARTING


> Loading data...
> OK
> Creating trainset...
> OK
> Training...
> OK 		 It Took:  0 seconds


In [30]:
from surprise import dump
import os
model_filename = "./model.pickle"
print (">> Starting dump")
# Dump algorithm and reload it.
file_name = os.path.expanduser(model_filename)
dump.dump(file_name, algo=algo)
print (">> Dump done")
print(model_filename)

>> Starting dump
>> Dump done
./model.pickle


In [31]:
def load_model(model_filename):
    print (">> Loading dump")
    from surprise import dump
    import os
    file_name = os.path.expanduser(model_filename)
    _, loaded_model = dump.load(file_name)
    print (">> Loaded dump")
    return loaded_model

In [32]:
from pprint import pprint as pp
model_filename = "./model.pickle"
def itemRating(user, item):
    uid = str(user)
    iid = str(item) 
    loaded_model = load_model(model_filename)
    prediction = loaded_model.predict(user, item, verbose=True)
    rating = prediction.est
    details = prediction.details
    uid = prediction.uid
    iid = prediction.iid
    true = prediction.r_ui
    ret = {
        'user': user, 
        'item': item, 
        'rating': rating, 
        'details': details,
        'uid': uid,
        'iid': iid,
        'true': true
        }
    pp (ret)
    print ('\n\n')
    return ret
print(itemRating(user = "610", item = "10"))

>> Loading dump
>> Loaded dump
user: 610        item: 10         r_ui = None   est = 5.00   {'was_impossible': False}
{'details': {'was_impossible': False},
 'iid': '10',
 'item': '10',
 'rating': 5,
 'true': None,
 'uid': '610',
 'user': '610'}



{'user': '610', 'item': '10', 'rating': 5, 'details': {'was_impossible': False}, 'uid': '610', 'iid': '10', 'true': None}


In [33]:
itemRating('Pinocchio', 7.4)

>> Loading dump
>> Loaded dump
user: Pinocchio  item: 7.4        r_ui = None   est = 5.00   {'was_impossible': False}
{'details': {'was_impossible': False},
 'iid': 7.4,
 'item': 7.4,
 'rating': 5,
 'true': None,
 'uid': 'Pinocchio',
 'user': 'Pinocchio'}





{'user': 'Pinocchio',
 'item': 7.4,
 'rating': 5,
 'details': {'was_impossible': False},
 'uid': 'Pinocchio',
 'iid': 7.4,
 'true': None}